In [ ]:
# Copyright 2023, Acadential, All rights reserved.

# 16-12. BERT 모델을 활용한 NLP 프로젝트

In [1]:
# Install transformers, datasets libraries
!pip install transformers datasets

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/98/46/f6a79f944d5c7763a9bc13b2aa6ac72daf43a6551f5fb03bccf0a9c2fec1/transformers-4.33.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 715.9 kB/s eta 0:00:001m761.2 kB/s eta 0:00:01
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/aa/f3/3fc97336a0e90516901befd4f500f08d691034d387406fdbde85bea827cc/huggingface_hub-0.17.3-py3-none-any.whl.metadata
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/03/5e/9a4cabe86a3b4e67bd2cf795a2e84de01c735c8c1c1d88795425847ccbbe/regex-2023.8.8-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━

# Google Drive을 Colab에 Mount하기

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Lesson/인프런 강의 - 딥러닝 이론 실무 완전 정복/practicals/section_16/")

In [2]:
import transformers 

In [3]:
from transformers.models.bert import (BertModel,
                                      BertConfig,
                                      BertForSequenceClassification,
                                      BertTokenizer)

# IMDB Dataset

RNN 실습에서 구현하였던 코드를 사용해서 IMDB 데이터셋을 불러오겠습니다.

In [4]:
from src.data import get_imdb_dataset, get_tokenizer, tokenize_dataset, get_dataloader

In [5]:
# Dataset 불러오기
dataset = get_imdb_dataset()

# Tokenizer 생성
tokenizer = get_tokenizer()

# Tokenizer를 이용해 Dataset을 Tokenize
tokenized_dataset = tokenize_dataset(dataset, tokenizer)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [7]:
train_dataloader, test_dataloader = get_dataloader(tokenized_dataset)

# BERT 모델 정의하기
IMDB 데이터셋은 Sentiment Analysis로 Sequence Classification task의 한 종류입니다. \
따라서 Sequence Classification 용 BERT 모델을 정의하고, 이를 학습시켜보겠습니다.

In [8]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
sample_data = next(iter(train_dataloader))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
sample_data.keys()

dict_keys(['label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [12]:
output = model(input_ids=sample_data["input_ids"],
               attention_mask=sample_data["attention_mask"],
               token_type_ids=sample_data["token_type_ids"],
               labels=sample_data["label"],
               return_dict=True)

In [13]:
type(output)

transformers.modeling_outputs.SequenceClassifierOutput

In [14]:
output

SequenceClassifierOutput(loss=tensor(0.7069, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0490,  0.2314],
        [-0.1440,  0.2051],
        [-0.0839,  0.1474],
        [ 0.0355,  0.1337],
        [-0.1383,  0.1994],
        [-0.1206,  0.1635],
        [-0.2156,  0.2050],
        [-0.0634,  0.1834],
        [-0.0970,  0.1848],
        [-0.0833,  0.1902],
        [-0.1903,  0.1976],
        [-0.1709,  0.1961],
        [-0.2238,  0.1675],
        [-0.0439,  0.1164],
        [-0.1793,  0.1604],
        [-0.0841,  0.1478]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
from tqdm import tqdm 
import torch 

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

best_test_acc = 0

train_loss_list = []
test_loss_list = []
test_acc_list = []
train_acc_list = []

for epoch in range(10):
    correct = 0 
    model.train()
    size = len(train_dataloader.dataset) 
    tbar = tqdm(train_dataloader)
    
    for batch in tbar:
        optimizer.zero_grad()
        
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        label = batch["label"].to(device)
        output = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=label,
                    return_dict=True)
        
        loss = output.loss 
        loss.backward()
        optimizer.step()
        
        tbar.set_description(f"loss - {loss.item():.3f}")
        train_loss_list.append(loss.item())
        
        pred = output.logits 
        
        correct += (pred.argmax(1) == label).type(torch.float).sum().item()

    correct /= size
    train_acc_list.append(correct)

    correct = 0
    test_loss = 0
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    
    tbar = tqdm(test_dataloader)
    model.eval()
    
    for batch in tbar:
        with torch.no_grad():
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            label = batch["label"].to(device)
            output = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=label,
                        return_dict=True)
            loss = output.loss 
            pred = output.logits 
        
            correct += (pred.argmax(1) == label).type(torch.float).sum().item()
            tbar.set_description(f'loss - {loss.item():.3f}')
    
    test_loss /= num_batches
    correct /= size

    test_loss_list.append(test_loss)
    test_acc_list.append(correct)
    print(f"test loss   {test_loss}")
    print(f"test acc   {correct}")
    
    if correct > best_test_acc:
        best_test_acc = correct
        os.makedirs('checkpoints', exist_ok=True)
        torch.save(model.state_dict(), 'checkpoints/best_bert_model_pretrained.pt')


In [30]:
import pandas as pd 
import os 

os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(train_loss_list)
plt.xlabel('Iteration')
plt.ylabel("Train Loss")
plt.title("BERT (pretrained) - Train Loss vs Iteration")
plt.savefig('figures/bert_train_loss_pretrained.png')

In [ ]:
plt.figure()
plt.plot(train_acc_list)
plt.xlabel('Epochs')
plt.ylabel("Train Accuracy")
plt.title("BERT (pretrained) - Train accuracy vs epochs")
plt.savefig('figures/bert_train_acc_pretrained.png')

In [ ]:
plt.figure()
plt.plot(test_loss_list)
plt.xlabel('Epochs')
plt.ylabel("Test Loss")
plt.title("BERT (pretrained) - Test Loss vs epochs")
plt.savefig('figures/bert_test_loss_pretrained.png')

In [ ]:
plt.figure()
plt.plot(test_acc_list)
plt.xlabel('Epochs')
plt.ylabel("Test Accuracy")
plt.title("BERT (pretrained) - Test accuracy vs epochs")
plt.savefig('figures/bert_test_acc_pretrained.png')

In [ ]:
train_loss_summary = {"bert_pretrained_": train_loss_list}
train_acc_summary = {"bert_pretrained": train_acc_list}
test_loss_summary = {"bert_pretrained": test_loss_list}
test_acc_summary = {"bert_pretrained": test_acc_list}

In [ ]:
pd.DataFrame(train_loss_summary).to_csv('results/train_loss_summary.csv')
pd.DataFrame(train_acc_summary).to_csv('results/train_acc_summary.csv')
pd.DataFrame(test_loss_summary).to_csv('results/test_loss_summary.csv')
pd.DataFrame(test_acc_summary).to_csv('results/test_acc_summary.csv')

# Randomly Initialized BERT

In [3]:
from transformers import BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertForSequenceClassification(configuration)

In [ ]:
from src.train_val import train_loop, val_loop
import torch 

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

best_test_acc = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

train_loss_list = []
test_loss_list = []
test_acc_list = []
train_acc_list = []

for epoch in range(10):
    train_loss, train_acc = train_loop(model, train_dataloader, optimizer)
    train_loss_list.extend(train_loss)
    train_acc_list.append(train_acc)
    
    test_loss, test_acc = val_loop(model, test_dataloader)
    print(f"test loss   {test_loss}")
    print(f"test acc   {test_acc}")
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        os.makedirs('checkpoints', exist_ok=True)
        torch.save(model.state_dict(), 'checkpoints/best_bert_model_random.pt')

In [ ]:
train_loss_summary["bert_random"] = train_loss_list
train_acc_summary["bert_random"] = train_acc_list
test_loss_summary["bert_random"] = test_loss_list
test_acc_summary["bert_pretrained"] = test_acc_list

In [ ]:
pd.DataFrame(train_loss_summary).to_csv('results/train_loss_summary.csv')
pd.DataFrame(train_acc_summary).to_csv('results/train_acc_summary.csv')
pd.DataFrame(test_loss_summary).to_csv('results/test_loss_summary.csv')
pd.DataFrame(test_acc_summary).to_csv('results/test_acc_summary.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(train_loss_list)
plt.xlabel('Iteration')
plt.ylabel("Train Loss")
plt.title("BERT (random) - Train Loss vs Iteration")
plt.savefig('figures/bert_train_loss_random.png')

In [ ]:
plt.figure()
plt.plot(train_acc_list)
plt.xlabel('Epochs')
plt.ylabel("Train Accuracy")
plt.title("BERT (random) - Train accuracy vs epochs")
plt.savefig('figures/bert_train_acc_random.png')

In [ ]:
plt.figure()
plt.plot(test_loss_list)
plt.xlabel('Epochs')
plt.ylabel("Test Loss")
plt.title("BERT (random) - Test Loss vs epochs")
plt.savefig('figures/bert_test_loss_random.png')

In [ ]:
plt.figure()
plt.plot(test_acc_list)
plt.xlabel('Epochs')
plt.ylabel("Test Accuracy")
plt.title("BERT (pretrained) - Test accuracy vs epochs")
plt.savefig('figures/bert_test_acc_random.png')

# Comparison between BERT pretrained vs. BERT random initialized

In [ ]:
plt.figure()
plt.plot(train_acc_summary["bert_random"], label="Random")
plt.plot(train_acc_summary["bert_pretrained"], label="Pretrained")
plt.xlabel('Epochs')
plt.ylabel("Train Accuracy")
plt.legend()
plt.title("BERT - Train accuracy vs epochs")
plt.savefig('figures/bert_train_acc.png')

In [ ]:
plt.figure()
plt.plot(test_loss_summary["bert_random"], label="Random")
plt.plot(test_loss_summary["bert_pretrained"], label="Pretrained")
plt.xlabel('Epochs')
plt.ylabel("Test Loss")
plt.legend()
plt.title("BERT - Test Loss vs epochs")
plt.savefig('figures/bert_test_loss.png')

In [ ]:
plt.figure()
plt.plot(test_acc_summary["bert_random"], label="Random")
plt.plot(test_acc_summary["bert_pretrained"], label="Pretrained")
plt.xlabel('Epochs')
plt.ylabel("Test Accuracy")
plt.legend()
plt.title("BERT - Test accuracy vs epochs")
plt.savefig('figures/bert_test_acc.png')